This notebook will consider the bounce task as mirroring the location with respect to the border. Input will be a coordinate out of bounds.

In [1]:
import pandas as pd
import numpy as np

### Straight forward

In [2]:
df = pd.DataFrame(columns=["x","x_vel","y","y_vel"]) #,"x0","x1","y0","y1"])
def add(row, df):
    """Docstring"""
    d = dict()
    cols = list(df.columns)
    for i in np.arange(len(cols)):
        d[cols[i]] = row[i]
    return df.append(d, ignore_index=True)

def add_rnd(df):
    x_vel = np.sign(np.random.rand() - 0.5) * np.random.rand() * 0.1
    y_vel = np.sign(np.random.rand() - 0.5) * np.random.rand() * 0.1
    x = np.random.rand()
    y = np.random.rand()
    return add([x,x_vel,y,y_vel],df) #,0,1,0,1], df)

def add_next(df):
    x,x_vel,y,y_vel = list(df.iloc[-1])
    return add([x+x_vel,x_vel,y+y_vel,y_vel], df)

In [3]:
df = pd.DataFrame(columns=["x","x_vel","y","y_vel"]) #,"x0","x1","y0","y1"])
for i in range(100000):
    if i % 10 == 0: df = add_rnd(df)
    else: df = add_next(df)

In [4]:
df.to_csv('sim_data2.csv', index=False)

### Bounces (mirroring)
X-axis only. Border only at x = 1

In [13]:
df = pd.DataFrame(columns=["x","x_vel","y","y_vel"])
for i in range(2000):
    x_vel = np.random.rand() * 0.1
    y_vel = np.random.rand() * 0.1
    x = 1 + np.random.uniform(low=0, high=np.abs(x_vel))
    y = np.random.rand()
    df = add([x,x_vel,y,y_vel],df)
    df = add([2-x,-x_vel,y,y_vel],df)

df.to_csv('bounces_mirror.csv', index=False)

In [14]:
df.head()

,x,x_vel,y,y_vel
0,1.026408,0.070358,0.246548,0.090114
1,0.973592,-0.070358,0.246548,0.090114
2,1.050825,0.057095,0.647579,0.059419
3,0.949175,-0.057095,0.647579,0.059419
4,1.077612,0.098985,0.079573,0.044087


### Attention

Is there going to be a bounce? Bool

In [12]:
df = pd.DataFrame(columns=["x","y","is_bounce"])
for i in range(2000):
    x_vel = np.random.rand() * np.sign(np.random.rand()-0.5) * 0.1
    x = 1 + np.random.uniform(low=-np.abs(x_vel), high=np.abs(x_vel))
    y = np.random.rand()
    boolean = (x > 1)
    df = add([x,y,boolean],df)

df.to_csv('bounces_att.csv', index=False)